#### Warm-up
#### The number of comments posted per year will likely trend upward over time as more users join Reddit. However, the popularity of some subreddits may increase or decrease over time. Find An example of both.

#### Loading dataset

In [1]:
# from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import from_json, col
conf = SparkConf().setAppName('FirstSpark2').setMaster('Spark')
sc = SparkContext.getOrCreate()

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df = sqlContext.read.json("hdfs://orion11:20001/sample_sampled_reddit/")

print(type(df))
print(df.count())
df.show(1)

<class 'pyspark.sql.dataframe.DataFrame'>
30926243
+--------+--------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+---------+--------------+-------+------------+-----+-----+------------+--------+---------------+------------+---+------------+
|archived|        author|author_cakeday|author_flair_css_class|author_flair_text|                body|body_html|controversiality|created|created_utc|distinguished|downs|edited|gilded|     id|  link_id|mod_reports|name|parent_id|removal_reason|replies|retrieved_on|saved|score|score_hidden|stickied|      subreddit|subreddit_id|ups|user_reports|
+--------+--------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+---------+--------------+-------+--------

#### Filtering dataset

In [2]:
df2 = df.filter(
~(df.body.like('[deleted]'))
    & ~(df.body.isNull())
    & ~(df.author.like('[deleted]'))
    & ~(df.author.like('AutoModerator')) 
    & ~(df.author.rlike("[bB][oO][tT]"))

)

print(type(df2))
print(df2.count())
df2.show(1)

<class 'pyspark.sql.dataframe.DataFrame'>
27303462
+--------+--------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+---------+--------------+-------+------------+-----+-----+------------+--------+---------------+------------+---+------------+
|archived|        author|author_cakeday|author_flair_css_class|author_flair_text|                body|body_html|controversiality|created|created_utc|distinguished|downs|edited|gilded|     id|  link_id|mod_reports|name|parent_id|removal_reason|replies|retrieved_on|saved|score|score_hidden|stickied|      subreddit|subreddit_id|ups|user_reports|
+--------+--------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+---------+--------------+-------+--------

#### Creating a Year Column based on created_utc column in dataset

In [3]:
from pyspark.sql.functions import year, month, dayofmonth, from_unixtime
from pyspark.sql.types import DateType

df3 = (df2
      .withColumn("year", year(from_unixtime("created_utc").cast(DateType())))
#       .withColumn("month",month(from_unixtime("created_utc").cast(DateType())))
      )
df3.show(2)

+--------+--------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+---------+--------------+-------+------------+-----+-----+------------+--------+---------------+------------+---+------------+----+
|archived|        author|author_cakeday|author_flair_css_class|author_flair_text|                body|body_html|controversiality|created|created_utc|distinguished|downs|edited|gilded|     id|  link_id|mod_reports|name|parent_id|removal_reason|replies|retrieved_on|saved|score|score_hidden|stickied|      subreddit|subreddit_id|ups|user_reports|year|
+--------+--------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+---------+--------------+-------+------------+-----+-----+------------+--------+--

#### Group by Subreddit

In [7]:
df4 = df3.groupBy(df3.subreddit, df3.year)
df4 = df4.count().orderBy('count', ascending=False)

print(type(df4))
print(df4.count())
df4.show(15)

<class 'pyspark.sql.dataframe.DataFrame'>
187565
+---------------+----+------+
|      subreddit|year| count|
+---------------+----+------+
|      AskReddit|2016|507143|
|      AskReddit|2015|458307|
|      AskReddit|2014|425660|
|      AskReddit|2013|400348|
|      AskReddit|2012|285988|
|       politics|2016|174268|
|      AskReddit|2011|140039|
|      AskReddit|2017|134778|
|          funny|2013|118633|
|leagueoflegends|2015|114906|
|          funny|2012|107919|
|     The_Donald|2016|101635|
|          funny|2014|101407|
|leagueoflegends|2014| 96791|
|           pics|2013| 90135|
+---------------+----+------+
only showing top 15 rows



#### An example of subreddit where popularity is decreasing

we check for subreddit is 'pics', count of comments on 'pics'.

Number of comments peak at year 2013 and is on decline since.
We see a huge jump in year 2010 when Instagram came around.

In [9]:
df5 = df4.where(df4.subreddit.like('pics')).orderBy(df4.year, ascending=False)
df5.show()

+---------+----+-----+
|subreddit|year|count|
+---------+----+-----+
|     pics|2017|17639|
|     pics|2016|59963|
|     pics|2015|70732|
|     pics|2014|75620|
|     pics|2013|90135|
|     pics|2012|81051|
|     pics|2011|70335|
|     pics|2010|31765|
|     pics|2009| 8109|
|     pics|2008| 2945|
+---------+----+-----+



#### An example of subreddit where popularity is increasing
we check for subreddit = 'videos'
We see that few years after 'pics' were on rise, comments on 'videos' started incresing .
Count of comments saw huge jump in year 2012. Can we attributed to introduction of videos and such in instagram after getting acquired by facebook.

In [10]:
df5 = df4.where(df4.subreddit.like('funny')).orderBy(df4.year, ascending=False)
df5.show()

+---------+----+-----+
|subreddit|year|count|
+---------+----+-----+
|   videos|2017|13191|
|   videos|2016|56055|
|   videos|2015|61333|
|   videos|2014|54883|
|   videos|2013|44455|
|   videos|2012|31858|
|   videos|2011|16403|
|   videos|2010| 3078|
|   videos|2009| 1066|
|   videos|2008|  329|
+---------+----+-----+

